In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## sys_micrograd

In [2]:
class Value:
    
    def __init__(self, data, _children=(), _op=(), label=''):
        self.data  = data
        self._prev = set(_children)
        self._op = _op
        self.label = label
        self.grad = 0.0 
        self._backward = lambda: None

    def __repr__(self):
        return f"Value(data={self.data})"
    
    # Operations
    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value( self.data + other.data , (self, other), '+') 
        
        def _backward(): # by addition our job is to propogate __add__'s out's grad into self and other 
                         # i.e pass it backward (the out.grad backwards to the 2 parents) other and self
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward
        
        return out
    
    def __radd__(self, other):
        return self + other
    
    def __neg__(self): 
        return self * -1
    
    def __sub__(self, other):
        return self + (-other)
    
    def __truediv__(self, other): # self/other
        return self * other**-1
        
    def __rmul__(self, other): # for the case of -- other * self
        return self * other
    
    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value( self.data * other.data, (self, other), '*' ) 
          
        def _backward(): # by multiplication our job is to multiply the weight of the other variable (if we are talking about self we need other, if its other we need self)
                            # and we multiply this (ther other variables data) by the out.grad
                            # this is 
            self.grad  += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward    
    
        return out
    
    
    def __pow__(self, other):
        assert isinstance(other, (int, float)), "We only support int or float data types for now"
        out = Value( self.data ** other, (self,), f'**{other}' ) 
        
        def _backward(): 
            self.grad += (other * ( self.data ** (other-1)) )* out.grad # other = n, and self.data = n
        out._backward = _backward
        
        return out
    
    
    def exp(self):
        x = self.data
        out = Value(math.exp(x), (self, ), 'exp')
        
        def _backward():
            self.grad += math.exp(x) * out.grad
        out._backward = _backward
        
        return out
           
    
    def tanh(self):
        x = self.data
        t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
        out = Value(t, (self, ), 'tanh')
        
        
        def _backward(): # this is simply the derivative of the tanh function, multiplied by the out.grad 
                            # since this is the chain rule 
            self.grad = (1 - t**2) * out.grad
        out._backward = _backward
        
        return out
    
    
    def backward(self):
        topo = []
        visited = set()
        # only adds a node once that nodes the children are added to the list
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)

        build_topo(self)
        topo
        self.grad = 1.0
        # now we call the 
        for node in reversed(topo):
            node._backward()
    


In [3]:
# helper fucntions to visualize everything happening behind the scenes

from graphviz import Digraph

# Step 1 - build the connections/tree of operations and operands
def trace(root):
    # This builds a set of all nodes and edges in a graph
    nodes, edges = set(), set()
    
    # function to build everything
    def build(v):
        if v not in nodes:
            nodes.add(v)
        for child in v._prev:
            edges.add((child, v))
            build(child)
    
    # now use this to build the trace fro, the root
    build(root)
    return nodes, edges

# Step 2 - Draw the connections/tree of operations and operands
def draw_dot(root):
    dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'}) # LR - for left to right
    
    nodes, edges = trace(root)
    for _n in nodes:
        uid = str(id(_n))
        
        # for any value in the graph, create a rectangular ('record') node for it
        dot.node(name = uid, label="{ %s |  data %.4f | grad %.4f }" % (_n.label, _n.data, _n.grad ), shape='record')
        
        if _n._op:
            # a - if this value is a result of some operation, create an op node for it
            dot.node(name = uid + _n._op , label = _n._op)
            # b - and connec this node to it
            dot.edge(uid + _n._op, uid)
            
    
    for n1, n2 in edges:
        # connect n1 to the op node of n2
        dot.edge( str(id(n1)), str(id(n2)) + n2._op)
        
    return dot

In [78]:
# PART 3 - Backpropagation on a neuron

In [5]:
# inputs of the network : x1, x2
x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')

# weights of the network : w1, w2 
w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')

# bias of the neuron
b = Value(6.88137335870195432, label='b')

x1w1 = x1*w1 ; x1w1.label = "x1*w1";
x2w2 = x2*w2 ;  x2w2.label = "x2*w2";
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label = 'x1*w1 + x2*w2'

n = x1w1x2w2 + b
n.label = 'n' # neuron before activation function

o = n.tanh()
o.label = 'o'
o.backward()

In [4]:
import torch

# Tensors are n-dimensional arrays of scalars

x1 = torch.Tensor([2.0]).double() ;                   x1.requires_grad = True
x2 = torch.Tensor([0.0]).double() ;                   x2.requires_grad = True

w1 = torch.Tensor([-3.0]).double() ;                  w1.requires_grad = True
w2 = torch.Tensor([1.0]).double() ;                   w2.requires_grad = True

b = torch.Tensor([6.88137335870195432]).double() ;    b.requires_grad = True

n = x1*w1 + x2*w2 + b

o = torch.tanh(n)

print(o)
print(o.data.item())

o.backward()

print('---')
print('x2', x2.grad.item())
print('w2', w2.grad.item())
print('x1', x1.grad.item())
print('w1', w1.grad.item())

tensor([0.7071], dtype=torch.float64, grad_fn=<TanhBackward0>)
0.7071066904050358
---
x2 0.5000001283844369
w2 0.0
x1 -1.5000003851533106
w1 1.0000002567688737


In [6]:
# PART 4 - Building a Nerual net library - Multi Layer Perceptron

In [7]:
import random

In [24]:

# Define a single neuron : Each neuron has the (weights + bias) corresponding to the number of inputs (layer before)
class Neuron:
    
    def __init__(self, num_inputs):
        self.w = [Value(random.uniform(-1, 1)) for _ in range(num_inputs)]
        self.b = Value(random.uniform(-1, 1))
        
    def __call__(self, x ):
        # w * x + b
        #print(list(zip(self.w, x)))
        act =  sum( (wi*xi for wi, xi in zip(self.w, x)), self.b )
        out = act.tanh()
        return out
    
    def parameters(self):
        return self.w + [self.b]
    
    
    

# Define a layer : each layer is a list of neurons : where
#            - each neuron takes in the Num inputs, and 
#            - the layer itself consists of #nurons = number of outputs
class Layer:
    
    def __init__(self, num_input, num_output):
        self.neurons = [Neuron(num_input) for _ in range(num_output)] # this creates a num_input* numoutput matric where each num_input == neruron
    
    def __call__(self, x):
        outputs = [n(x) for n in self.neurons]
        return outputs[0] if len(outputs) == 1 else outputs # allows us to get a single value out if only one neuron
    
    def parameters(self):
        return [ neuron_param for neuron in self.neurons for neuron_param in neuron.parameters() ]
    
x = [2.0, 3.0]
n = Neuron(2)
print(n(x))

print("\nstarting layers")
m = Layer(2, 3)
print( m(x) )


Value(data=-0.6685172492330929)

starting layers
[Value(data=-0.9912934225210936), Value(data=0.993694666619432), Value(data=-0.9588773377941429)]


In [25]:
class MLP: # Multi layer perceptron
    
    def __init__(self, num_inputs, num_outputs):
        sz = [num_inputs] + num_outputs # 
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(num_outputs))]

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x
    
    def parameters(self):
        return [ layer_params for layer in self.layers for layer_params in layer.parameters() ]

#draw_dot(n(x))

Value(data=0.3709180895659462)

In [229]:
x = [2.0, 3.0, -1.0] # 3d input
n = MLP(3, [4,4,1]) # 3 inputs to 2 layers of 4, with one output (l1 = 3,4, l2= 4,4, l3 = 4, 1 )
n(x)

Value(data=-0.9234811842778495)

In [234]:
xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0]
]
ys = [1.0, -1.0, -1.0, 1.0] # targets

ypred = [ n(x) for x in xs ]
ypred

[Value(data=0.9333489696995945),
 Value(data=-0.952580516638162),
 Value(data=-0.9304558153122353),
 Value(data=0.9182652432820448)]

In [279]:
# 5 - to imporve the network we want to calculte the MSE for each prediction to use to update the network properly

#  A - create an automated (a) forward pass a (b) backward pass , and thenn (c) the update (gradiant decent)
for epochs in range(20):
    
    # (a) : forward pass
    ypred = [ n(x) for x in xs ]
    loss = sum( (y_pred - y_ground_truth)**2 for y_ground_truth, y_pred in zip(ys,ypred) ) # mean_squared_error
    
    # (b) backward pass 
    # WARNING *** must reset the grads to zero, so we only accumulate the grads for this loss.backward, not prev runs
    #.      if we dont -- our steps are huge
    for p in n.parameters():
        p.grad = 0.0
    loss.backward()
    
    # (c) the update
    # 6 - nudge the values of the weights and biases, based on the gradiant decent
    #        --> gradiant points in the direction of greater loss -> so we want to point the grad in the oppposite dir
    for p in n.parameters():
        p.data += (-0.05) * p.grad
        
        
    print(epochs, loss)

0 Value(data=0.0013085057056744435)
1 Value(data=0.0013057155511955246)
2 Value(data=0.001302937195502468)
3 Value(data=0.0013001705641180925)
4 Value(data=0.0012974155831897548)
5 Value(data=0.0012946721794827348)
6 Value(data=0.0012919402803738419)
7 Value(data=0.00128921981384507)
8 Value(data=0.0012865107084772673)
9 Value(data=0.0012838128934439439)
10 Value(data=0.0012811262985051837)
11 Value(data=0.0012784508540015123)
12 Value(data=0.001275786490848019)
13 Value(data=0.0012731331405283531)
14 Value(data=0.0012704907350889715)
15 Value(data=0.0012678592071333475)
16 Value(data=0.0012652384898162437)
17 Value(data=0.0012626285168381801)
18 Value(data=0.0012600292224397946)
19 Value(data=0.0012574405413964056)


In [278]:
# 7 - now we 
ypred

[Value(data=0.98233105112955),
 Value(data=-0.9857414684229721),
 Value(data=-0.981526439539894),
 Value(data=0.9786801073864201)]